In [ ]:
import numpy as np
import pandas as pd

from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
output_notebook()

from ipywidgets import interact

from collections import OrderedDict
old_settings = np.seterr(over='ignore') #Ignore warnings about overflow data points

In [ ]:
# Set up constant
N = 200;
kB = 8.617e-5; #eV/K
T = 0.001; # Boltzmann constant measured interms of chemical potential
currenT = [str(T) + ' K']*N; #Create an array of labels to initiate interactive legend
kBT = kB * T;
E_min = 0;
E_max = 10.0; # Energy in eV
factor = 0.2;

# Set up the variable space for 1D plot
En_mu = np.linspace(E_min, E_max, N) #Energy measured in units of chemical potential
f = 1.0/(np.exp( (factor*En_mu-1)/kBT )+1) # Occupation number according to F-D statistics
df = np.gradient(f) # Take the first derivative of the F-D distribution.

In [ ]:
# # Set up variable space for 2D plot
# ex = ey = np.linspace(-E_max,E_max,2*N)
# Ex,Ey = np.meshgrid(ex, ey)
# Er = np.sqrt(Ex**2 + Ey**2)
# ff = 1.0/(np.exp( (factor*Er-1)/kBT )+1)

In [ ]:
# Set up a static 1D plot
plot = figure(plot_height=400, plot_width=400, title="Fermi-Dirac statistics in 1D",
              tools="pan,reset,save,wheel_zoom",
              x_range=[0.0, 10.0], y_range=[0.0, 1.1])
plot.xaxis[0].axis_label='Energy'
plot.yaxis[0].axis_label='Fermi-Dirac distribution (f)'
r = plot.line(En_mu, f, legend = "0.001 K", line_width=3, line_alpha=0.6)

In [ ]:
# Set up an interactive 1D plot
plot1 = figure(plot_height=400, plot_width=400, title="Fermi-Dirac statistics in 1D",
              tools="pan,reset,save,wheel_zoom",
              x_range=[0.0, 10.0], y_range=[0.0, 1.1])
plot1.xaxis[0].axis_label='Energy'
plot1.yaxis[0].axis_label='Fermi-Dirac distribution (f)'
source1 = ColumnDataSource(dict(x=En_mu, y=f, label=currenT));
rrrrr = plot.line(x='x',y='y',legend='label', source=source1, line_color = 'red', line_width=3, line_alpha=0.6)

In [ ]:
# # Set up 2D plot #Commented out for future use
# plot2 = figure(plot_height=400, plot_width=400, title="Fermi-Dirac statistics in 2D",
#               tools="pan,reset,save,wheel_zoom",
#               x_range=[-10.0, 10.0], y_range=[-10.0, 10.0], tooltips=[("Ex", "$x"), ("Ey", "$y"), ("Probability", "@image")])
# plot2.xaxis[0].axis_label='Kx'
# plot2.yaxis[0].axis_label='Ky'
# rr = plot2.image(image = [ff], x=-10, y=-10, dw=20, dh=20, palette="Spectral11")

# #Optional black circle to mark the zero-temperature fermi level
# #rrr = plot2.circle(0.0, 0.0,alpha = 0, radius=1.0, level = "underlay")

In [ ]:
# Set up an interactive 1D plot of the first derivative of the F-D distribution
plot3 = figure(plot_height=400, plot_width=400, title="First derivative",
              tools="pan,reset,save,wheel_zoom",
              x_range=[0.0, 10.0], y_range=[0.0, 0.6])
plot3.xaxis[0].axis_label='Energy'
plot3.yaxis[0].axis_label='First derivative (f`)'
source3 = ColumnDataSource(dict(x=En_mu, y=-df, label=currenT));
rrr = plot3.line(x='x', y='y', legend = 'label', source=source3, line_color = 'red', line_width=3, line_alpha=0.6)

In [ ]:
# Set up a static 1D plot of the first derivative of the F-D distribution at low temperature
plot4 = figure(plot_height=400, plot_width=400, title="First derivative",
              tools="pan,reset,save,wheel_zoom",
              x_range=[0.0, 10.0], y_range=[0.0, 0.6])
plot4.xaxis[0].axis_label='Energy'
plot4.yaxis[0].axis_label='First derivative (f`)'
rrrrrr = plot3.line(En_mu, -df, legend = "0.001 K", line_width=3, line_alpha=0.6)

In [ ]:
# Set up callbacks to update the 1D live graph
def update_data(T):
    # Generate the new curve
    kBT = kB*T;
    currenT = [str(T) + ' K']*N;
    En_mu = np.linspace(E_min, E_max, N) #Energy measured in units of chemical potential
    rrrrr.data_source.data['y'] = 1.0/(np.exp( (factor*En_mu-1)/kBT )+1);
    rrrrr.data_source.data['label'] = currenT;
    Ex,Ey = np.meshgrid(np.linspace(-E_max,E_max,2*N), np.linspace(-E_max,E_max,2*N));
    Er = np.sqrt(Ex**2 + Ey**2);
#     rr.data_source.data['image'] = [1.0/(np.exp( (factor*Er-1)/kBT )+1)]    
    rrr.data_source.data['y'] = -np.gradient(rrrrr.data_source.data['y']);
    rrr.data_source.data['label'] = currenT;
    push_notebook();

In [ ]:
show(row(plot,plot3),notebook_handle=True);
# show(column([row(plot, plot3), plot2]), notebook_handle=True)

In [ ]:
interact(update_data, T=(0.001,1000.001,2));